In [ ]:
import xarray as xr
ds = xr.open_dataset('pdsi.nc')
print(ds)

In [ ]:
import xarray as xr
import numpy as np
import csv
import pandas as pd

def get_pdsi_from_file(file_path, lat, lon, date):
    ds = xr.open_dataset(file_path)

    # Get the nearest latitude and longitude points from the dataset
    nearest_lat = ds['lat'].sel(lat=lat, method='nearest').values.item()
    nearest_lon = ds['lon'].sel(lon=lon, method='nearest').values.item()
    
   # Custom function to find the nearest date prior to the provided date
    def find_nearest_date(array, date):
        array = pd.to_datetime(array)
        idx = np.searchsorted(array, date, side='right') - 1
        return array[idx]

    
    date = pd.Timestamp(date)
    nearest_date = find_nearest_date(ds['day'].values, date)

    # Get the PDSI data for the nearest date and coordinates
    pdsi_data = ds['pdsi'].sel(day=nearest_date, lat=nearest_lat, lon=nearest_lon).values.item()

    if not np.isnan(pdsi_data):
        return pdsi_data
    else:
        return None


df = pd.read_csv("mapdata_wildfires.csv")


# List of coordinates and dates
coordinates_and_dates = []

for _, row in df.iterrows():
    lat = row['incident_latitude']
    lon = row['incident_longitude']
    date_created = row['incident_dateonly_created']
    
    # Create a tuple and append it to the list
    coordinates_and_dates.append((lat, lon, date_created))


file_path = 'pdsi.nc'

# CSV output file name
csv_output_file = 'drought_data_wildfires.csv'

# Fetch PDSI data for each coordinate and date and write to a CSV file
with open(csv_output_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Latitude', 'Longitude', 'Date', 'PDSI'])

    for coord_date in coordinates_and_dates:
        lat = coord_date[0]
        lon = coord_date[1]
        date = coord_date[2]

        pdsi = get_pdsi_from_file(file_path, lat, lon, date)
        csv_writer.writerow([lat, lon, date, pdsi])

print(f"PDSI data has been written to {csv_output_file}")
